In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import pandas as pd
from pathlib import Path
#import tensorflow as tf

# Import our input dataset
data = Path('./Data/merged_clean_df.csv')
merged_df = pd.read_csv(data)
merged_df

,reportedcity,dateofloss,elevationdifference,floodzone,latitude,longitude,numberoffloorsintheinsuredbuilding,occupancytype,originalconstructiondate,amountpaidonbuildingclaim,state,reportedzipcode,stns,obs,pred,stn_lat,stn_lon,new_date_column,geometry,rolling_total_obs
0,NO CHATHAM,2/21/2015,999,AE,41.7,-70.0,3,1,1/1/1955,2618.18,MA,2650,14281,0.38,0.000000,41.6875,-69.9933,2/21/2015,POINT (-69.9933 41.6875),3.800000e-01
1,NEW BRAUNFELS,10/30/2015,2,AE,29.7,-98.1,3,1,12/2/2003,3309.59,TX,78130,490,15.62,11.490905,29.6874,-98.6527,10/30/2015,POINT (-98.6527 29.6874),1.562000e+01
2,NEW BRAUNFELS,10/30/2015,7,AE,29.7,-98.1,2,1,7/1/2003,0.00,TX,78130,490,15.62,11.490905,29.6874,-98.6527,10/30/2015,POINT (-98.6527 29.6874),1.562000e+01
3,NEW BRAUNFELS,10/30/2015,0,C,29.7,-98.1,3,1,10/17/1998,67301.33,TX,78132,490,15.62,11.490905,29.6874,-98.6527,10/30/2015,POINT (-98.6527 29.6874),1.562000e+01
4,NEW BRAUNFELS,10/30/2015,999,C,29.7,-98.1,3,3,7/1/1995,0.00,TX,78130,490,15.62,11.490905,29.6874,-98.6527,10/30/2015,POINT (-98.6527 29.6874),1.562000e+01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52172,WEST COLUMBIA,10/13/2015,2,A20,34.0,-81.1,2,1,9/1/1990,0.00,SC,29169,3961,0.00,0.000000,33.9970,-81.1290,10/13/2015,POINT (-81.129 33.997),6.200000e-13
52173,COLUMBUS,12/22/2015,999,A,32.5,-84.9,1,1,1/1/1973,5779.26,GA,31906,3739,4.98,5.187531,32.5181,-84.9378,12/22/2015,POINT (-84.9378 32.5181),4.980000e+00
52174,ELLIJAY,12/26/2015,999,X,34.7,-84.5,3,1,3/1/1989,0.00,GA,30540,5397,0.00,NaN,34.0565,-84.5853,12/26/2015,POINT (-84.5853 34.0565),5.830000e-13
52175,HAMMOND,4/11/2014,999,A,44.5,-75.6,2,1,1/1/1976,0.00,NY,13646,17845,0.32,0.000000,45.3167,-65.8833,4/11/2014,POINT (-65.88330000000001 45.3167),3.200000e-01


In [5]:
#convert date columns to datetime64[ns] 
merged_df['originalconstructiondate'] =  pd.to_datetime(merged_df['originalconstructiondate'], format='%Y/%m/%d', errors='ignore')
merged_df['dateofloss'] =  pd.to_datetime(merged_df['dateofloss'], format='%Y/%m/%d', errors='ignore')

#merged_df.dtypes
#split the dates into sperate columns for dd, mm and yyyy
#split date of loss
merged_df[['lossyear','lossmonth','lossday']] = merged_df.dateofloss.apply(lambda x: pd.Series(x.strftime("%Y,%m,%d").split(",")))

#split original construction date
merged_df[['builtyear','builtmonth','builtday']] = merged_df.originalconstructiondate.apply(lambda x: pd.Series(x.strftime("%Y,%m,%d").split(",")))


#convert split columns to floats
merged_df['lossyear'] = merged_df['lossyear'].astype(float)
merged_df['lossmonth'] = merged_df['lossmonth'].astype(float)
merged_df['builtyear'] = merged_df['builtyear'].astype(float)

#age of the property at the time of loss
merged_df['propertyage'] = merged_df['lossyear'] - merged_df['builtyear'] 

AttributeError: 'str' object has no attribute 'strftime'

In [ ]:
merged_df.head(3)

In [ ]:
#keeping chosen features and dropping all other columns from merged data

df = merged_df.copy()
df = df.drop("reportedzipcode", axis=1)
df = df.drop("dateofloss", axis=1)
df = df.drop("elevationdifference", axis=1)
df = df.drop("latitude", axis=1)
df = df.drop("longitude", axis=1)
df = df.drop("numberoffloorsintheinsuredbuilding", axis=1)
df = df.drop("originalconstructiondate", axis=1)
df = df.drop("reportedcity", axis=1)
df = df.drop("lossyear", axis=1)
df = df.drop("lossday", axis=1)
df = df.drop("builtyear", axis=1)
df = df.drop("builtmonth", axis=1)
df = df.drop("builtday", axis=1)
df = df.drop("stn_lat", axis=1)
df = df.drop("stn_lon", axis=1)
df = df.drop("geometry", axis=1)
df = df.drop("new_date_column", axis=1)

#dropping columns that might be providing redundant information
df = df.drop("obs", axis=1)
df = df.drop("pred", axis=1)
df = df.drop("stns", axis=1)


#df.head(3)
FEMA_encoded = pd.get_dummies(df, columns=["state", "floodzone", "occupancytype", "lossmonth"])

In [ ]:
#generating a correlation matrix
corrMatrix = df.corr()

import seaborn as sn
sn.heatmap(corrMatrix, annot=True)

In [ ]:
FEMA_encoded.columns.tolist()

In [ ]:
acc_rain = FEMA_encoded.apply(lambda x: True if x['rolling_total_obs'] < 0.1 else False , axis=1)
acc_rainrows = len(acc_rain[acc_rain == True].index)
print('Outliner Elevations <0 : ', acc_rainrows)

NoPayout = FEMA_encoded.apply(lambda x: True if x['amountpaidonbuildingclaim'] <= 0 else False , axis=1)
NoPayoutRows = len(NoPayout[NoPayout == True].index)
print('Payouts <0 : ', NoPayoutRows)

In [ ]:
FEMA_encoded.rolling_total_obs.min()

In [ ]:
acc_rain = FEMA_encoded.apply(lambda x: True if x['rolling_total_obs'] > 0 else False , axis=1)
acc_rainrows = len(acc_rain[acc_rain == True].index)
print('Outliner Elevations <0 : ', acc_rainrows)

In [ ]:
FEMA_encoded_withrain = FEMA_encoded.copy()

indexNames = FEMA_encoded_withrain[FEMA_encoded_withrain['rolling_total_obs'] < 0.5 ].index
 
# Delete these row indexes from dataFrame
FEMA_encoded_withrain.drop(indexNames , inplace=True)

FEMA_encoded_withrain.rolling_total_obs.min()

In [ ]:
y =FEMA_encoded_withrain.copy()
X = FEMA_encoded_withrain.copy()
X = X.drop("amountpaidonbuildingclaim", axis=1)

In [ ]:
# Split the X and y into X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,
                                                   y['amountpaidonbuildingclaim'],
                                                   random_state=1)

In [ ]:
# Train the Classifier

from sklearn.linear_model import LinearRegression

#random forest, neural networks with linear regression
model = LinearRegression() 
model.fit(X_train, y_train)

In [ ]:
score_with_rain= model.score(X_test, y_test)
score_with_rain

In [ ]:
FEMA_zones = FEMA_encoded_withrain.copy()

In [ ]:
filter_zones_A = [col for col in FEMA_zones if col.startswith('floodzone_A')]
filter_zones_V = [col for col in FEMA_zones if col.startswith('floodzone_V')]

FEMA_zones[filter_zones_V]

In [ ]:
FEMA_zones["Zone A"] = FEMA_zones[filter_zones_A].sum(axis = 1, skipna = True)
FEMA_zones["Zone V"] = FEMA_zones[filter_zones_V].sum(axis = 1, skipna = True) 

FEMA_zones

In [ ]:
FEMA_zones = FEMA_zones.drop(FEMA_zones[filter_zones_A], axis=1)

In [ ]:
FEMA_zones = FEMA_zones.drop(FEMA_zones[filter_zones_V], axis=1)

In [ ]:
FEMA_zones.columns.tolist()

In [ ]:
FEMA_zones.rename(columns={"Zone A": "floodzone_A", "Zone V": "floodzone_V"})

In [ ]:
y =FEMA_zones.copy()
X =FEMA_zones.copy()
X = X.drop("amountpaidonbuildingclaim", axis=1)

In [ ]:
# Split the X and y into X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,
                                                   y['amountpaidonbuildingclaim'],
                                                   random_state=1)

In [ ]:
# Train the Classifier

from sklearn.linear_model import LinearRegression

#random forest, neural networks with linear regression
model = LinearRegression() 
model.fit(X_train, y_train)

In [ ]:
score_with_combinedzones= model.score(X_test, y_test)
score_with_combinedzones

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
import numpy as np
df_pred = pd.DataFrame({'Actual Payoffs': y_test, 'Predicted Payoffs': y_pred})
df_pred

In [ ]:
df_pred["off by (%)"] = (df_pred["Actual Payoffs"]*100)/df_pred["Predicted Payoffs"]
df_pred

In [ ]:
FEMA_encoded_withrain = FEMA_encoded.copy()

indexNames = FEMA_encoded_withrain[FEMA_encoded_withrain['rolling_total_obs'] < 0.5 ].index
 
# Delete these row indexes from dataFrame
FEMA_encoded_withrain.drop(indexNames , inplace=True)

FEMA_encoded_withrain.rolling_total_obs.min()